# Embedding approximation and PCA initialisation using **imitation**

This is a demonstration of **imitation loss**.
This approach initialises the ViVAE model parameters according to an existing, trained DR model, effectively biasing the embedding process.
Through computing L2 distances between points embedded by the ViVAE model's encoder and their corresponding embedding by the reference model, we quickly optimise the embedding to match the desired starting point before proceeding to train ViVAE fully.
We then continue with the rest of the training procedure.

If we are training our VAE on PCA-reduced data, we can apply a trivial pre-training using the embedding obtained from taking the top 2 principal components, which we obtain by taking the first two dimensions of the input.
This makes the process very fast.

<hr>

In this case study, we PCA-initialise and train a decoder-less ViVAE model to create a 2-dimensional embedding of the [*Reed*](https://cellxgene.cziscience.com/collections/48259aa8-f168-4bf5-b797-af8e88da6637) Human Breast Atlas immune cells dataset.

<hr>

## **0.** Set-up

First, let's import required libraries and load in data.

In [ ]:
import numpy as np
import torch
import ViVAE
import ViScore
import matplotlib.pyplot as plt

The data below is downloaded from CELLxGENE and pre-processed, as described [here](https://github.com/saeyslab/ViScore/tree/main/benchmarking) and in our manuscript.

In [ ]:
input          = np.load('../../ViScore/benchmarking/data/Reed_input.npy', allow_pickle=True)
input_denoised = np.load('../../ViScore/benchmarking/data/Reed_input_denoised.npy', allow_pickle=True)
labs           = np.load('../../ViScore/benchmarking/data/Reed_labels.npy', allow_pickle=True)

The extraction of top 2 principal components from input data is implemented in the `FirstNDimsExtractor` class in ViVAE.
**This assumes that the input consists of PCA-reduced data, with columns ordered by decreasing PC eigenvalue.**

In [ ]:
from ViVAE.losses import FirstNDimsExtractor
ref_model = FirstNDimsExtractor(latent_dim=2)

## **1.** Pre-training

We will now create a ViVAE model and pre-train it to imitate the projection of data obtained from taking the first 2 principal components.
To keep things fast, we do not train a decoder (`lam_recon=0.`).

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
model = ViVAE.ViVAE(input_dim=input.shape[1], random_state=42)
model.fit(X=input_denoised, n_epochs=10, batch_size=2048, lam_recon=0., lam_kldiv=1., lam_mds=0., lam_imit=1., ref_model=ref_model)

Let's verify that the embedding obtained by the pre-trained ViVAE model and low-dimensional PCA are indeed similar.

In [ ]:
fig_pc, ax_pc = ViVAE.plot_embedding(input_denoised[:,range(2)], labs, show=False, figsize=(4,4))
ax_pc.get_legend().remove()
ax_pc.set_title('Top 2 PCs')
fig_pc.savefig('imitation_pc.png', dpi=600)

In [ ]:
fig_pc

In [ ]:
emb0 = model.transform(input_denoised)
fig0, ax0 = ViVAE.plot_embedding(emb0, labs, show=False, figsize=(4,4))
ax0.get_legend().remove()
ax0.set_title('ViVAE imitating top 2 PCs')
fig0.savefig('imitation_vivae0.png', dpi=600)

In [ ]:
fig0

## **2.** Continued training

We can now continue the training process by optimising stochastic-MDS along with KL divergence from latent prior.

In [ ]:
model.fit(X=input_denoised, n_epochs=20, batch_size=2048, lam_recon=0., lam_kldiv=1., lam_mds=100., lam_imit=0., ref_model=ref_model)

In [ ]:
emb1 = model.transform(input_denoised)
fig1, ax1 = ViVAE.plot_embedding(emb1, labs, show=False, figsize=(4,4))
ax1.get_legend().remove()
ax1.set_title('Fully trained ViVAE')
fig1.savefig('imitation_vivae1.png', dpi=600)

In [ ]:
fig1